#01 Import Libraries

In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os

#02 Import Data

In [2]:
#define path to main Instacart project folder
InstacartProjectFolder=r'C:\Users\user\Instacart Basket Analysis 02042023'

In [4]:
#import orders data sets
ords_prods_merge=pd.read_pickle(os.path.join(InstacartProjectFolder,'02 Data','Prepared Data','orders_new_variables.pkl'))

In [5]:
# Question 2 - find aggregated mean for order number on complete data set
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#0 Question 3 - The output from the entire data set is different from the subset in the following ways
- every department id from 1 to 21 is included in the list
- the average values for oder_number have changed

Both of these are due to more rows of data contributing to the calculation

#0 Question 4 - Loyalty flags on complete data set

In [6]:
# step 1 - add new column using transform() function
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [9]:
# step 2 - add flag for loyal customers
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [10]:
# step 3 - add flag for regular customers
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [11]:
# step 4 - add flag for new customers
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
#step 5, show counts for new column
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

#0 Question 5 - find descriptive statistics for price on each loyalty category

In [14]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

Based on this output, loyal customers tend to purchase the cheapest products. Perhaps they know how to watch for sales.

#0 Question 6 - Create Spending Flag

In [24]:
# step 1 - add new column using transform() function
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [25]:
# step 2 - add flag for low spenders
ords_prods_merge.loc[ords_prods_merge['avg_price'] <10, 'spending_flag'] = 'low spender'

In [26]:
# step 3 - add flag for high spenders
ords_prods_merge.loc[ords_prods_merge['avg_price'] >=10, 'spending_flag'] = 'high spender'

In [27]:
#step 4 check counts for each flag
ords_prods_merge['spending_flag'].value_counts(dropna = False)

low spender     31770614
high spender      634245
Name: spending_flag, dtype: int64

#0 Question 7 - Create order frequency flag

In [28]:
# step 1 - add new column using transform() function
ords_prods_merge['median_days_last_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [29]:
# step 2 - add flag for non-frequent customer
ords_prods_merge.loc[ords_prods_merge['median_days_last_order'] > 20, 'frequency_flag'] = 'non-frequent customer'

In [30]:
# step 3 - add flag for regular customers
ords_prods_merge.loc[(ords_prods_merge['median_days_last_order'] > 10) & (ords_prods_merge['median_days_last_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [31]:
# step 4 - add flag for frequent customer
ords_prods_merge.loc[ords_prods_merge['median_days_last_order']  <=10, 'frequency_flag'] = 'frequent customer'

In [32]:
#step 5 - check counts for each flag
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequent customer        21559853
Regular customer          7208564
non-frequent customer     3636437
NaN                             5
Name: frequency_flag, dtype: int64

In [33]:
ords_prods_merge['median_days_last_order'].value_counts(dropna = False)

7.0     5682399
6.0     2983604
8.0     2713964
5.0     2486061
4.0     2314289
         ...   
26.5       5514
29.5       5095
1.5        1684
0.5         235
NaN           5
Name: median_days_last_order, Length: 62, dtype: int64

In [34]:
ords_prods_merge['median_days_last_order'].shape

(32404859,)

In [35]:
ords_prods_merge['days_since_prior_order'].value_counts(dropna = False)

7.0     3476198
30.0    3067374
6.0     2517498
5.0     2124434
4.0     2078648
NaN     2076096
8.0     1931945
3.0     1876188
2.0     1463724
9.0     1217841
14.0    1029656
10.0     967187
1.0      940474
13.0     851114
11.0     820761
12.0     774164
15.0     671458
16.0     461335
0.0      448329
21.0     444142
17.0     382114
20.0     379732
18.0     350587
19.0     334882
22.0     308849
28.0     253151
23.0     225834
27.0     204061
24.0     193421
25.0     180417
26.0     177775
29.0     175470
Name: days_since_prior_order, dtype: int64

#0 - observation on frequency flag - there are 5 rows displaying NaN, which matches 5 rows in the newly created column median_days_last_order.  I am unsure why these 5 rows earn this value, perhaps it is because they have a NaN in the original days_since_prior_order column.  It seems my logic to create this flag is correct, and only 5 bad values would mean I could drop these from my analysis if necessary.

#0 Question 9 - export as pickle file

In [36]:
ords_prods_merge.to_pickle(os.path.join(InstacartProjectFolder, '02 Data','Prepared Data', 'orders_with_flags.pkl'))